In [ ]:
import sys
sys.path.append("..")

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets
from models.CNN2D import CNN2D

In [ ]:
model = CNN2D()
model.compile(loss=['mse','mse','mse','mse'],
              loss_weights=[0.25,0.25,0.25,0.25],
              optimizer='adam',
              metrics=['mse','mse','mse','mse'])
# Temporarily use cifar10 images before real images dataloader is available
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Dummy metadata
train_metadata = np.random.uniform(size=(train_images.shape[0], 5))
test_metadata = np.random.uniform(size=(test_images.shape[0], 5))

# Create dummy outputs, should be GHI at t0, t0 + 1, t0 + 3 and t0 + 6
train_labels = np.array([(x, x+5, 2*x, x+100) for x in train_labels])
test_labels = np.array([(x, x+5, 2*x, x+100) for x in test_labels])

# Build model with inputs shapes to initialize weights
model.build([(None, 32, 32, 3), (None, 5)])
print(model.summary())

# Fit model with firt 5000 examples
history = model.fit([train_images[:5000], train_metadata[:5000]],
                    [train_labels[:5000, 0],
                     train_labels[:5000, 1],
                     train_labels[:5000, 2],
                     train_labels[:5000, 3]],
                    epochs=5,
                    validation_data=([test_images, test_metadata],
                                     [test_labels[:, 0],
                                      test_labels[:, 1],
                                      test_labels[:, 2],
                                      test_labels[:, 3]]))

In [ ]:
plt.plot(history.history['output_1_mse'], label='t0_mse')
plt.plot(history.history['output_2_mse'], label='t1_mse')
plt.plot(history.history['output_3_mse'], label='t2_mse')
plt.plot(history.history['output_4_mse'], label='t3_mse')

plt.xlabel('Epoch')
plt.ylabel('train MSE')
plt.ylim(0, 100)
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['val_output_1_mse'], label='t0_mse')
plt.plot(history.history['val_output_2_mse'], label='t1_mse')
plt.plot(history.history['val_output_3_mse'], label='t2_mse')
plt.plot(history.history['val_output_4_mse'], label='t3_mse')

plt.xlabel('Epoch')
plt.ylabel('val MSE')
plt.legend(loc='lower right')

In [ ]:
test_loss = model.evaluate([test_images, test_metadata],
                                     [test_labels[:, 0],
                                      test_labels[:, 1],
                                      test_labels[:, 2],
                                      test_labels[:, 3]],
                                     verbose=2)
print(test_loss)